In [11]:
%run using_func.ipynb

結論
===
1. Q4 投入學習時間低於 Q3 許多, 推測與疫情有很大關聯, 也還許多原因, 例如 : 寒暑假、畢業季、在家時間長短
2. 性別無太大影響到學習情況
3. 照學科分類，文商類別在高投入時間上，明顯少於其他類組許多，可能是教材對此類組相對較難，也許可以新增教材內容，或增加練習
4. 照學習目標分類，Q3、Q4中，Tourist、Career Switcher的組成比例發生很大的變化
5. 照學習目標來看繳交率，此兩類型的學生有很大的差距，可能想轉職的學生，有比較大的動機去寫作業，或者Tourist發現這不是他想學的內容、難度太高，導致繳交率下降
6. 照時間來看，沒甚麼區別，疫情雖然影響到人數，但沒影響到繳交率，還是學生本身的學習目標為主
7. 教學難度對於Career Switcher學生會是轉職必要的階段，對Tourist學生不同動機則造成了學習意願下降
8. 推測原本的教學無法同時適用於Career Switcher與Tourist學生，而且這個問題已存在半年以上

# 學生投入學習的時間

## 1.Q4 班級投入時間與 Q3 班級投入時間是否真的有落差?

In [12]:
conn = conn_login()
cursor = conn.cursor()

cursor.execute('''
  WITH count_班級人數 AS (
  SELECT `班級`, COUNT(`班級`) AS `班級總人數`
  FROM table1_students
  GROUP BY `班級`
)

SELECT t1.`班級`,
       SUM(t3.`學習時間（分鐘）`)/60.0 AS `總學習時間(小時)`,
       count_班級人數.`班級總人數`,
       (SUM(t3.`學習時間（分鐘）`)/60.0/count_班級人數.`班級總人數`) AS `平均每人學習時間`
FROM table3_sessions t3
JOIN table1_students t1 ON t1.id = t3.student_id
JOIN count_班級人數 ON count_班級人數.`班級` = t1.`班級`
GROUP BY t1.`班級`
ORDER BY CAST(SUBSTRING(t1.`班級`, 1, 2) AS UNSIGNED)
''')
res = cursor.fetchall()
fetchall_TO_DF(res)

,班級,總學習時間(小時),班級總人數,平均每人學習時間
0,7月班,26621.7833,412,64.61597896
1,8月班,28787.5167,436,66.02641437
2,9月班,32905.3167,519,63.40138086
3,10月班,17346.3000,444,39.06824324
4,11月班,20790.0333,503,41.33207422
5,12月班,19524.4167,478,40.84605997


## 2.高投入學生的人數佔比，在 Q3 班 和 Q4 班是否有顯著差異？


In [13]:
cursor.execute('''
WITH high_student AS (
  SELECT `student_id`, SUM(`學習時間（分鐘）`/60.0) AS `學習時間`
  FROM table3_sessions
  GROUP BY `student_id`
),
count_班級人數 AS (
  SELECT `班級`, COUNT(`班級`) AS `班級總人數`
  FROM table1_students
  GROUP BY `班級`
)

SELECT t1.`班級`,
       COUNT(t1.`班級`) AS `高投入人數`,
       count_班級人數.`班級總人數`,
       (COUNT(t1.`班級`) * 100.0 / count_班級人數.`班級總人數`)  AS `高投入占比(%)`
FROM table1_students t1
JOIN high_student hs ON hs.`student_id` = t1.`id`
JOIN count_班級人數 ON count_班級人數.`班級` = t1.`班級`
WHERE hs.`學習時間` > 70
GROUP BY t1.`班級`
ORDER BY CAST(SUBSTRING(t1.`班級`, 1, 2) AS UNSIGNED)
''')
res = cursor.fetchall()
fetchall_TO_DF(res)

,班級,高投入人數,班級總人數,高投入占比(%)
0,7月班,187,412,45.38835
1,8月班,194,436,44.49541
2,9月班,214,519,41.23314
3,10月班,12,444,2.70270
4,11月班,12,503,2.38569
5,12月班,11,478,2.30126


# 驗證不同背景的學生，是否會有不同的投入程度

## 1.找出在 Q3、Q4 中，不同性別學生的學習投入狀況?

In [14]:
cursor.execute('''
WITH learntime AS (
  SELECT student_id, SUM(`學習時間（分鐘）`)/60.0 AS `學習時間`
  FROM table3_sessions
  GROUP BY student_id
),
gender AS (
  SELECT 性別, COUNT(student_id) AS `總人數`
  FROM table2_demographic
  GROUP BY 性別
),
hs AS (
SELECT t2.`性別`,
       gender.`總人數`,
       COUNT(`學習時間`) AS `高投入人數`,
       COUNT(`學習時間`) * 100.0 / gender.`總人數` AS `高投入占比(%)`
FROM learntime
JOIN table2_demographic t2 ON t2.student_id = learntime.student_id
JOIN gender ON gender.性別 = t2.`性別`
WHERE `學習時間` > 70
GROUP BY t2.`性別`
),
ls AS (
SELECT t2.`性別`,
       gender.`總人數`,
       COUNT(`學習時間`) AS `低投入人數`,
       COUNT(`學習時間`) * 100.0 / gender.`總人數` AS `低投入占比(%)`
FROM learntime
JOIN table2_demographic t2 ON t2.student_id = learntime.student_id
JOIN gender ON gender.性別 = t2.`性別`
WHERE `學習時間` < 25
GROUP BY t2.`性別`
)

SELECT *
FROM (
  SELECT hs.`性別`,
         hs.`總人數`,
         hs.`高投入人數`,
         hs.`高投入占比(%)`,
         ls.`低投入人數`,
         ls.`低投入占比(%)`
  FROM hs
  JOIN ls ON hs.`性別` = ls.`性別`
  UNION
  SELECT '總計',
         SUM(hs.`總人數`) as `總人數`,
         SUM(hs.`高投入人數`) as `高投入人數`,
         SUM(hs.`高投入人數`) * 100.0 / (SELECT SUM(`總人數`) FROM gender) AS `高投入占比(%)`,
         SUM(ls.`低投入人數`) as `低投入人數`,
         SUM(ls.`低投入人數`) * 100.0 / (SELECT SUM(`總人數`) FROM gender) AS `低投入占比(%)`
  FROM hs
  JOIN ls ON hs.`性別` = ls.`性別`
) AS res
ORDER BY
  CASE res.`性別`
    WHEN '男' THEN 1
    WHEN '女' THEN 2
    WHEN '不提供' THEN 3
    ELSE 4
  END;
''')
res = cursor.fetchall()
fetchall_TO_DF(res)

,性別,總人數,高投入人數,高投入占比(%),低投入人數,低投入占比(%)
0,男,1299,302,23.24865,210,16.16628
1,女,1380,311,22.53623,228,16.52174
2,不提供,113,17,15.04425,15,13.27434
3,總計,2792,630,22.56447,453,16.22493


## 2.分析其他學生背景因素對於學習投入的影響?

## 3.學科背景

In [15]:
cursor.execute('''
WITH learntime AS (
  SELECT student_id, SUM(`學習時間（分鐘）`)/60.0 AS `學習時間`
  FROM table3_sessions
  GROUP BY student_id
),
g1 AS (
  SELECT lt.student_id, lt.學習時間
  FROM table2_demographic t2
  JOIN learntime lt ON lt.student_id = t2.student_id
  WHERE t2.學科背景 IN ('文', '商')
),
g2 AS (
  SELECT lt.student_id, lt.學習時間
  FROM table2_demographic t2
  JOIN learntime lt ON lt.student_id = t2.student_id
  WHERE t2.學科背景 IN ('理論科學','工程科學')
),
g3 AS (
  SELECT lt.student_id, lt.學習時間
  FROM table2_demographic t2
  JOIN learntime lt ON lt.student_id = t2.student_id
  WHERE t2.學科背景 IN ('資訊科學（資工、資管等等）')
),
g4 AS (
  SELECT lt.student_id, lt.學習時間
  FROM table2_demographic t2
  JOIN learntime lt ON lt.student_id = t2.student_id
  WHERE t2.學科背景 IN ('其他')
)
-- G2示範, 透過where去控制高投入、低投入
SELECT COUNT(*) AS `總人數`
FROM g2
WHERE g2.學習時間 < 25;

''')
res = cursor.fetchall()
fetchall_TO_DF(res)

,總人數
0,125


In [16]:
# 依照想搜尋的資料改變table、where條件
dictres = {'學科背景': ['文+商', '理論 + 工程', '資訊', '其他', '總計'],
           '總人數': [1096, 1155, 338, 203 , 0],
           '高投入人數': [104, 387, 97, 42, 0],
           '低投入人數': [267, 125, 28, 33, 0]
           }
res = pd.DataFrame(dictres)
res.iloc[[4],1:4] = res.iloc[:4,1:4].sum()

res['高投入占比(%)'] = round(res['高投入人數'] * 100 / res['總人數'], 2)
res['低投入占比(%)'] = round(res['低投入人數'] * 100 / res['總人數'], 2)
res = res.reindex(columns=['學科背景', '總人數', '高投入人數', '高投入占比(%)', '低投入人數', '低投入占比(%)'])
res

,學科背景,總人數,高投入人數,高投入占比(%),低投入人數,低投入占比(%)
0,文+商,1096,104,9.49,267,24.36
1,理論 + 工程,1155,387,33.51,125,10.82
2,資訊,338,97,28.70,28,8.28
3,其他,203,42,20.69,33,16.26
4,總計,2792,630,22.56,453,16.22


## 4.學習目標

In [17]:
cursor.execute('''
    SELECT t2.學習目標, COUNT(t2.student_id) as `sum`
    from table2_demographic t2
    JOIN table1_students t1 ON t2.student_id = t1.id
    where 班級 IN ('10月班', '11月班', '12月班')
    GROUP BY `學習目標`;
''')
res = cursor.fetchall()
fetchall_TO_DF(res)

,學習目標,sum
0,Learner,372
1,Tourist,650
2,Career Switcher,403


In [18]:
# 依照想搜尋的資料改變where條件
dictres = {'學習目標': ['Career Switcher', 'Leaner', 'Tourist', '總計'],
           'Q3': [688, 304, 375, 0],
           'Q4': [403, 372, 650, 0]
           }
res = pd.DataFrame(dictres)
res.iloc[[3],1:3] = res.iloc[:3,1:3].sum()

res['Q3占比(%)'] = round(res['Q3'] * 100 / res.iloc[3,1], 2)
res['Q4占比(%)'] = round(res['Q4'] * 100 / res.iloc[3,2], 2)
res = res.reindex(columns=['學習目標', 'Q3', 'Q3占比(%)', 'Q4', 'Q4占比(%)'])
res

,學習目標,Q3,Q3占比(%),Q4,Q4占比(%)
0,Career Switcher,688,50.33,403,28.28
1,Leaner,304,22.24,372,26.11
2,Tourist,375,27.43,650,45.61
3,總計,1367,100.00,1425,100.00


# 驗證假設學習成效是否遇到挑戰

## 1.計算 Q4 班 Tourist vs Career Switcher 的繳交率

In [19]:
cursor.execute('''
with hw AS (
SELECT student_id, count(*) as `作業繳交數`
FROM table4_submissions
GROUP by student_id
)
SELECT t2.`學習目標`,
	SUM(hw.`作業繳交數`) as `作業繳交數`,
       count(t2.student_id) * 15 as `應繳交數`,
       count(t2.student_id) as `人數`
FROM table2_demographic t2
join table1_students t1 ON t1.id = t2.student_id
left join hw ON t2.student_id = hw.student_id
-- where `班級` in ('7月班', '8月班', '9月班')
where `班級` in ('10月班', '11月班', '12月班')
group by `學習目標`
''')

res = cursor.fetchall()
Q4 = fetchall_TO_DF(res)

cursor.execute('''
with hw AS (
SELECT student_id, count(*) as `作業繳交數`
FROM table4_submissions
GROUP by student_id
)
SELECT t2.`學習目標`,
	SUM(hw.`作業繳交數`) as `作業繳交數`,
       count(t2.student_id) * 15 as `應繳交數`,
       count(t2.student_id) as `人數`
FROM table2_demographic t2
join table1_students t1 ON t1.id = t2.student_id
left join hw ON t2.student_id = hw.student_id
where `班級` in ('7月班', '8月班', '9月班')
group by `學習目標`
''')
res = cursor.fetchall()
Q3 = fetchall_TO_DF(res)

In [20]:
# 轉換型態，原本是object
Q3['作業繳交數'] = Q3['作業繳交數'].astype('int64')
Q4['作業繳交數'] = Q4['作業繳交數'].astype('int64')

Q3['繳交作業率(%)'] = round(Q3['作業繳交數'] * 100 / Q3['應繳交數'], 2)
Q4['繳交作業率(%)'] = round(Q4['作業繳交數'] * 100 / Q4['應繳交數'], 2)
print('Q4 繳交情況')
display(Q4)

print('Q3 繳交情況')
display(Q3)

Q4 繳交情況


,學習目標,作業繳交數,應繳交數,人數,繳交作業率(%)
0,Learner,3451,5580,372,61.85
1,Tourist,3726,9750,650,38.22
2,Career Switcher,4932,6045,403,81.59


Q3 繳交情況


,學習目標,作業繳交數,應繳交數,人數,繳交作業率(%)
0,Learner,2942,4560,304,64.52
1,Tourist,2033,5625,375,36.14
2,Career Switcher,8614,10320,688,83.47


## 2.Q4 班 Tourist 的學生在哪種作業類型遇到挑戰？不同類型的作業完成率為何？

In [21]:
cursor.execute('''
SELECT 作業類型, count(作業類型)
from table5_assignments
GROUP by 作業類型
''')

res = cursor.fetchall()
concept = res[0][1]
practice = res[1][1]

In [22]:
cursor.execute('''
SELECT t5.作業類型, count(assignment_id) as `實際繳交數`
FROM table1_students t1
left join table2_demographic t2 on t1.id = t2.student_id
left join table4_submissions t4 on t1.id = t4.student_id
left join table5_assignments t5 on t4.assignment_id = t5.id
where `班級` in ('10月班', '11月班', '12月班') and `學習目標` = 'Tourist'
GROUP by 作業類型
''')
res = cursor.fetchall()
Q4_T = fetchall_TO_DF(res)

cursor.execute('''
SELECT t5.作業類型, count(assignment_id) as `實際繳交數`
FROM table1_students t1
left join table2_demographic t2 on t1.id = t2.student_id
left join table4_submissions t4 on t1.id = t4.student_id
left join table5_assignments t5 on t4.assignment_id = t5.id
where `班級` in ('10月班', '11月班', '12月班') and `學習目標` = 'Career Switcher'
GROUP by 作業類型
''')
res = cursor.fetchall()
Q4_S = fetchall_TO_DF(res)

In [23]:
# 人數在上一個環節有求出, Q4_T = 650
Q4_T_number = 650
Q4_T['應繳交數'] = [Q4_T_number * practice, Q4_T_number * concept]
Q4_T['作業繳交率(%)'] = round(Q4_T['實際繳交數'] * 100 / Q4_T['應繳交數'], 2)

Q4_S_number = 403
Q4_S['應繳交數'] = [Q4_S_number * practice, Q4_S_number * concept]
Q4_S['作業繳交率(%)'] = round(Q4_S['實際繳交數'] * 100 / Q4_S['應繳交數'], 2)

res = pd.concat([Q4_T, Q4_S], axis=1, keys = ['Q4_Tourist', 'Q4_Career Switcher'])
res


Q4_Tourist                      Q4_Career Switcher                     
        作業類型 實際繳交數  應繳交數 作業繳交率(%)               作業類型 實際繳交數  應繳交數 作業繳交率(%)
0         實作   994  3900    25.49                 實作  1885  2418    77.96
1         觀念  2732  5850    46.70                 觀念  3047  3627    84.01

## 3.Q4 班 Tourist 的學生在哪種作業難度遇到挑戰？

In [24]:
cursor.execute('''
SELECT 作業難度, count(作業難度)
from table5_assignments
GROUP by 作業難度;
''')

res = cursor.fetchall()
easy = res[0][1]
medium = res[1][1]
hard = res[2][1]

In [25]:
cursor.execute('''
SELECT t5.作業難度, count(assignment_id) as `實際繳交數`
FROM table1_students t1
left join table2_demographic t2 on t1.id = t2.student_id
left join table4_submissions t4 on t1.id = t4.student_id
left join table5_assignments t5 on t4.assignment_id = t5.id
where `班級` in ('10月班', '11月班', '12月班') and `學習目標` = 'Tourist'
GROUP by 作業難度
''')
res = cursor.fetchall()
Q4_T = fetchall_TO_DF(res)

cursor.execute('''
SELECT t5.作業難度, count(assignment_id) as `實際繳交數`
FROM table1_students t1
left join table2_demographic t2 on t1.id = t2.student_id
left join table4_submissions t4 on t1.id = t4.student_id
left join table5_assignments t5 on t4.assignment_id = t5.id
where `班級` in ('10月班', '11月班', '12月班') and `學習目標` = 'Career Switcher'
GROUP by 作業難度
''')
res = cursor.fetchall()
Q4_S = fetchall_TO_DF(res)

In [26]:
display(Q4_T)
# 補沒寫的
Q4_T.loc[2] = ['實務挑戰題', 0]

,作業難度,實際繳交數
0,隨堂練習,2764
1,綜合應用,962


In [27]:
# 人數在上一個環節有求出, Q4_T = 650
Q4_T_number = 650
Q4_T['應繳交數'] = [Q4_T_number * easy, Q4_T_number * medium, Q4_T_number * hard]
Q4_T['作業繳交率(%)'] = round(Q4_T['實際繳交數'] * 100 / Q4_T['應繳交數'], 2)

Q4_S_number = 403
Q4_S['應繳交數'] = [Q4_S_number * easy, Q4_S_number * medium, Q4_S_number * hard]
Q4_S['作業繳交率(%)'] = round(Q4_S['實際繳交數'] * 100 / Q4_S['應繳交數'], 2)

res = pd.concat([Q4_T, Q4_S], axis=1, keys = ['Q4_Tourist', 'Q4_Career Switcher'])
res


Q4_Tourist                      Q4_Career Switcher                     
        作業難度 實際繳交數  應繳交數 作業繳交率(%)               作業難度 實際繳交數  應繳交數 作業繳交率(%)
0       隨堂練習  2764  3900    70.87               隨堂練習  2398  2418    99.17
1       綜合應用   962  3900    24.67               綜合應用  2184  2418    90.32
2      實務挑戰題     0  1950     0.00              實務挑戰題   350  1209    28.95

In [28]:
cursor.close()
conn.close()